# **ResNet 50 from scratch**

In [ ]:
#@title **Loading data from gdrive to memory**
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/ANNDL_Project

%cd ../../datasets

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/15O-hdeXPBICW_RJDpxFM_YoVERTpVMjK/ANNDL_Project


In [ ]:
!yes A | unzip data_splitted.zip -d data_splitted

In [ ]:
#@title **Imports**
import warnings
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_uniform
from keras.models import Model
from keras.optimizers import Adam
import numpy as np
import os
import random
import pandas as pd
import scipy
from datetime import datetime
from google.colab import files

In [ ]:
#@title **Metadata and variables**

tfk = tf.keras
tfkl = tf.keras.layers

dir = "data_splitted"

input_shape = (96, 96, 3)
nclasses = 8
seed = 42


In [ ]:
#@title **Setting seed and/or suppressing warnings**
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Setting random seed for reproducibility
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
#@title **Method that returns ImageDataGenerators (iterators) of augmented images**
def get_dataset_generator(dir):
      
    train_dir = dir + "/train"
    test_dir = dir + "/test"
    val_dir = dir + "/val"


    train_data_gen = ImageDataGenerator(rotation_range=15,
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='reflect',
                                        )

    valid_data_gen = ImageDataGenerator(
        # rescale=1 / 255.
        )
    test_data_gen = ImageDataGenerator(
        # rescale=1 / 255.
        )

    train_gen = train_data_gen.flow_from_directory(directory=train_dir,
                                                           target_size=(input_shape[0], input_shape[1]),
                                                           color_mode='rgb',
                                                           class_mode='categorical',
                                                           batch_size=64,
                                                           shuffle=True)
    valid_gen = valid_data_gen.flow_from_directory(directory=val_dir,
                                                   target_size=(input_shape[0], input_shape[1]),
                                                   color_mode='rgb',
                                                   class_mode='categorical',
                                                   batch_size=64,
                                                   shuffle=True)
    test_gen = test_data_gen.flow_from_directory(directory=test_dir,
                                                 target_size=(input_shape[0], input_shape[1]),
                                                 color_mode='rgb',
                                                 class_mode='categorical',
                                                 batch_size=64,
                                                 shuffle=True)

    return train_gen, valid_gen, test_gen

In [ ]:
#@title **Utility function to create folders and callbacks for training**
def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('trained_models/citte')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%m-%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_dir + '/cp-{val_accuracy:.2f}-{epoch:02d}.ckpt', # Checkpoint is saved with validation accuracy in the filename
                                                     monitor='val_accuracy', 
                                                     save_weights_only=True, # True to save only weights
                                                     save_best_only=True, # True to save only the best epoch 
                                                     initial_value_threshold=0.7
                                                     ) # Model is saved only if val_accuracy > initial_value_threshold

  callbacks.append(ckpt_callback)


  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks, exp_dir



---

# **Models from scratch**

In [ ]:
#@title **2. ResNet-50**
def identity_block(X, f, filters, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
        
    X = tfkl.Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfkl.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = tfkl.Activation('relu')(X)
        
    X = tfkl.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfkl.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = tfkl.Activation('relu')(X)

    X = tfkl.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfkl.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Add shortcut value to main path
    X = tfkl.Add()([X_shortcut, X])
    X = tfkl.Activation('relu')(X)
        
    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
        
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X
    X = tfkl.Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfkl.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = tfkl.Activation('relu')(X)
    X = tfkl.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfkl.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = tfkl.Activation('relu')(X)
    X = tfkl.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfkl.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    X_shortcut = tfkl.Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = tfkl.BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)
    X = tfkl.Add()([X_shortcut, X])
    X = tfkl.Activation('relu')(X)
   
    return X

def original_ResNet50(input_shape = (96, 96, 3), classes = 8):
    X_input = tfkl.Input(input_shape)
    X = tfkl.ZeroPadding2D((3, 3))(X_input)
    X = tfkl.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfkl.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = tfkl.Activation('relu')(X)
    X = tfkl.MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    X = tfkl.AveragePooling2D(pool_size=(2, 2),name='avg_pool')(X)
    X = tfkl.Flatten()(X)
    X = tfkl.Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    return model

def build_ResNet50(input_shape = (96, 96, 3), nclasses = 8):
    X_input = tfkl.Input(input_shape)
    X = tfkl.ZeroPadding2D((3, 3))(X_input)
    X = tfkl.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = tfkl.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = tfkl.Activation('relu')(X)
    X = tfkl.MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    X = tfkl.AveragePooling2D(pool_size=(2, 2),name='avg_pool')(X)
    X = tfkl.Flatten()(X)
    X = tfkl.Dropout(0.2, seed=seed)(X)
    X = tfkl.Dense(units=32, name='Classifier1', kernel_initializer=tfk.initializers.HeUniform(seed),
                                    activation='relu')(X)
    X = tfkl.Dense(nclasses, activation='softmax', name='fc' + str(nclasses), kernel_initializer = glorot_uniform(seed))(X)
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

    return model    

model = build_ResNet50(input_shape, 8)


In [ ]:
model.summary(expand_nested=True)

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 102, 102, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 48, 48, 64)   9472        ['zero_padding2d[0][0]']         
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, 48, 48, 64)   256         ['conv1[0][0]']                  
                                                                                           

In [ ]:
#@title Training of ResNet-50
epochs = 200
resnet_callbacks, model_folder_dir = create_folders_and_callbacks(model_name='ResNet50_From_Scratch_CB')

train_gen, val_gen, test_gen = get_dataset_generator(dir)

# Train the model
history = model.fit(
    x=train_gen,
    epochs=epochs,
    validation_data=val_gen,
    callbacks = resnet_callbacks
).history

Found 2475 images belonging to 8 classes.
Found 527 images belonging to 8 classes.
Found 540 images belonging to 8 classes.
Epoch 1/250
78/78 [==============================] - 16s 147ms/step - loss: 2.2223 - accuracy: 0.1911 - val_loss: 2.0523 - val_accuracy: 0.1461
Epoch 2/250
78/78 [==============================] - 12s 157ms/step - loss: 1.7665 - accuracy: 0.3301 - val_loss: 2.4272 - val_accuracy: 0.1461
Epoch 3/250
78/78 [==============================] - 11s 135ms/step - loss: 1.6525 - accuracy: 0.3782 - val_loss: 3.1443 - val_accuracy: 0.1518
Epoch 4/250
78/78 [==============================] - 11s 141ms/step - loss: 1.5601 - accuracy: 0.4117 - val_loss: 4.0207 - val_accuracy: 0.1518
Epoch 5/250
78/78 [==============================] - 10s 134ms/step - loss: 1.4926 - accuracy: 0.4356 - val_loss: 2.6041 - val_accuracy: 0.2068
Epoch 6/250
78/78 [==============================] - 11s 138ms/step - loss: 1.4566 - accuracy: 0.4533 - val_loss: 1.4678 - val_accuracy: 0.4535
Epoch 7/250



---

**Save of models**


In [ ]:
model.save("saved_models/...")



---

**Loading of models**

In [ ]:
#@title **Loading of an existing model (having only the weights)**
# after having already built the model...
tl_model.load_weights('saved_models/...')

In [ ]:
#@title **Loading of an existing model (Complete)**
model = tf.keras.models.load_model("saved_models/...")



---
**Model testing**


In [1]:
model_test_metrics = model.evaluate(test_gen, return_dict=True)

17/17 [==============================] - 1s 59ms/step - loss: 0.8071 - accuracy: 0.7396


In [ ]:
#@title **Testing a model and re-saving**

train_gen, val_gen, test_gen = get_dataset_generator(dir)

# saving model in saved_models/modelname_accuracy
saved_model_name = 'ResNet50_from_scratch_' + str(model_test_metrics["accuracy"])
saved_model_dir = 'saved_models/' + saved_model_name
ft_model.save(saved_model_dir)



---
**Download of the model in .zip format**


In [ ]:
zipped_model = saved_model_dir + '/' + saved_model_name + '.zip'

In [ ]:
!zip -r {zipped_model} {saved_model_dir}

In [ ]:
files.download(zipped_model)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



---

